# DREAMER dataset

In [ ]:
import numpy as np
import scipy
import pickle
import scipy.io
from sklearn import preprocessing
from argparse import ArgumentParser
from scipy import signal
import scipy.io as sio
#!pip install neurokit2
import neurokit2 as nk
import pandas as pd
import math
from sklearn.decomposition import PCA

In [ ]:
video_len = [25472,
16768,
44544,
21248,
17408,
24320,
24576,
50432,
18560,
8576,
12288,
23168,
47104,
21760,
39424,
24960,
32768,
23808]

# EEG Feature Extraction : feature_extraction_EEG
def preprocessing_and_feature_extraction_EEG(raw):#(file_name_csv,raw):
    EEG_feature = []
    for participant in range(0,23):
        for video in range(0,18):
            baselineLength = 61
            basl = np.zeros((128,14))
            baseline = raw['DREAMER'][0,0]['Data'][0,participant]['EEG'][0,0]['baseline'][0,0][video,0][:,:]
            for base_slot in range(0, int(baseline.shape[0]/128)): #calculate baseline
                basl+=baseline[base_slot*128:(base_slot+1)*128,:]
            basl /= 61
            stimuli = raw['DREAMER'][0,0]['Data'][0,participant]['EEG'][0,0]['stimuli'][0,0][video,0][:,:]
            for stim_slot in range(0,int(stimuli.shape[0]/128)):
                if stim_slot == 0:
                    stim = np.array(stimuli[stim_slot*128:(stim_slot+1)*128,:]-basl)
                else:
                    next_slot = stimuli[stim_slot*128:(stim_slot+1)*128,:]-basl
                    next_slot = np.array(next_slot)
                    stim = np.concatenate((stim, next_slot), axis=0)
            if video == 0:
                num_video = stim
            else:

                num_video = np.concatenate((num_video, stim), axis=0)

        EEG_feature.append(num_video)
    return EEG_feature

def Participants_Data(raw):
    # Create new dataframe with emotion, participant, and video data
    a=np.zeros((23,18,9),dtype=object)
    for participant in range(0,23):
        for video in range(0,18):
            a[participant,video,0]=raw['DREAMER'][0,0]['Data'][0,participant]['Age'][0][0][0]
            a[participant,video,1]=raw['DREAMER'][0,0]['Data'][0,participant]['Gender'][0][0][0]
            a[participant,video,2]=participant+1
            a[participant,video,3]=video+1
            a[participant,video,4]=['Searching for Bobby Fischer','D.O.A.', 'The Hangover', 'The Ring', '300',
                      'National Lampoon\'s VanWilder', 'Wall-E', 'Crash', 'My Girl', 'The Fly',
                      'Pride and Prejudice', 'Modern Times', 'Remember the Titans', 'Gentlemans Agreement',
                      'Psycho', 'The Bourne Identitiy', 'The Shawshank Redemption', 'The Departed'][video]
            a[participant,video,5]=['calmness', 'surprise', 'amusement', 'fear', 'excitement', 'disgust',
                      'happiness', 'anger', 'sadness', 'disgust', 'calmness', 'amusement',
                      'happiness', 'anger', 'fear', 'excitement', 'sadness', 'surprise'][video]
            a[participant,video,6]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreValence'][0,0][video,0].astype(float)
            a[participant,video,7]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreArousal'][0,0][video,0].astype(float)
            a[participant,video,8]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreDominance'][0,0][video,0].astype(float)
    b=pd.DataFrame(a.reshape((23*18,a.shape[2])),columns=['Age','Gender','Participant','Video','Video_Name','Target_Emotion','Valence','Arousal','Dominance'])
    ## combine feature extraction dataframes with the new dataframe
    #all_data=pd.concat([data_EEG,data_ECG,b],axis=1)
    return b

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
path = "./DREAMER.mat"

raw = sio.loadmat(path)
x = preprocessing_and_feature_extraction_EEG(raw)
x = np.array(x)
x = x.reshape((23*477184,x.shape[2]))
#y could be applied to all three categories labels
y = Participants_Data(raw)['Valence'] 
for i in range(len(y)):
  if y[i] < 4:
    y[i] = 0
  else:
    y[i] = 1

new_y = np.zeros(x.shape[0])
old_index = 0
new_index = 0
for people in range(23):
  for cur_len in video_len:
    for cur_sample in range(cur_len):
      new_y[new_index] = y[old_index]
      new_index += 1
    old_index += 1
print(x.shape, new_y.shape)
X = np.array(x)
new_y = np.array(new_y)

X_train, X_test, y_train, y_test = train_test_split(x, new_y, test_size=0.1, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(10975232, 14) (10975232,)
(9877708, 14) (1097524, 14) (9877708,) (1097524,)
